# ASP Interactive Fiction Generator

In [1]:
# HIDDEN

import re
from collections import defaultdict
from ipywidgets import Button, HBox, VBox, Layout, widgets
import ipywidgets
from IPython.display import display

In [2]:
# HIDDEN

class IF:
    """text"""
    
    def __init__(self, path, current = None, future = None):
        """text"""
        self.end = False
        if path: # first time
            self.choice = None
            self._create_tree(path)
        else: # recursive
            self._create_branch(current, future)
                            
    def _create_tree(self, path):
        """"""
        f = open(path)
        raw = f.read().split('State 0')[1:]
        
        next_states = defaultdict(lambda: [])
        
        for item in raw:
            path = item.split('State')
            path = self._clean_path(path)
            
            self.reactions = tuple(path[0])
            path = path[1:]
            
            next_states[tuple(path[0])].append(path[1:])
        
        self.children = [IF(None, state, next_states[state]) for state in next_states]
    
    def _clean_path(self, raw_path):
        """"""
        path = []
        for item in raw_path:
            item = item.replace('\n','')
            item = item.replace(':', '')
            item = ''.join([i for i in item if not i.isdigit()])
            item = item.strip(' ')
            item = item.split(' ')
            token = re.compile("\w+\(.*\)")
            for idx in range(len(item)):
                if token.match(item[idx]):
                    item[idx] = token.match(item[idx]).group()
                else:
                    item[idx] = None
            item = list(filter(None, item))
            path.append(item)
        return path
    
    def _create_branch(self, current, future):
        """"""        
        choice = re.compile('decision\(.*\)')
        self.choice = None
        self.reactions = []
        for item in current:
            if choice.match(item):
                self.choice = choice.match(item).group()
            else:
                self.reactions.append(item)
        
        next_states = defaultdict(lambda: [])   
            
        if future:
            for item in future:
                if len(item) > 1:
                    next_states[tuple(item[0])].append(item[1:])
                else:
                    next_states[tuple(item[0])] = None
            self.children = [IF(None, state, next_states[state]) for state in next_states]
        else:
            self.end = True
            self.children = None
            
    def play(self):
        for item in self.reactions:
            if(self.writable(item)):
                print(self.writable(item))
        choices = True
        if not self.children:
            choices = False
        else:
            for child in self.children:
                if not child.choice:
                    choices = False
        if choices == True:
            buttons = [Button(description=str(child.choice), 
                              layout=Layout(width='100%'))
                        for child in self.children]
            display(widgets.VBox(buttons))
            for idx in range(len(self.children)):
                buttons[idx].on_click(self.children[idx]._choose)
        else:
            #for item in self.reactions:
            #    print(str(item))
            if self.children:
                assert(len(self.children) == 1)
                self.children[0].play()
        
    def _choose(self, b):
        #for item in self.reactions:
        #    print(str(item))
        self.play()
        
    def writable(self, line):
        if 'role(' in line:
            return None
        return line
        
    def write_clean(self, path, tag = '', first = True):
        if first:
            f = open(path, 'w')
        with open(path, 'a') as f:
            for item in self.reactions:
                if self.writable(item):
                    f.write(self.writable(item) + '\n')
            
        pairs = []
        num = 0
        
        if self.children and len(self.children) != 1:
            for idx in range(len(self.children)):
                newtag = tag + str(num)
                num += 1
                with open(path, 'a') as f:
                    f.write(self.writable(self.children[idx].choice) + '|' + newtag + '\n')
                pairs.append((self.children[idx], newtag))
            for pair in pairs:
                with open(path, 'a') as f:
                    f.write(pair[1] + '\n')
                pair[0].write_clean(path, pair[1], False)
                
        elif self.children and len(self.children) == 1:
            tag = tag + str(num)
            with open(path, 'a') as f:
                f.write(tag + '\n')
            self.children[0].write_clean(path, tag, False)

# Testing

In [130]:
# HIDDEN

heist = IF('results/01-heist-19.txt')

In [136]:
# HIDDEN

#heist.write_clean('results/CLEAN-01-heist-19.txt')

In [131]:
# HIDDEN

heist.play()

setting(museum)
story(start)
has(character(guard),object(key))
is(character(guard),place(museum))
is(character(thief),place(museum))
is(object(painting),place(museum))
setting(museum)
story(rising)
has(character(guard),object(key))
is(character(guard),place(museum))
is(character(thief),place(museum))
is(object(painting),place(museum))


In [3]:
# HIDDEN

def process_output(path):
    '''takes path to ASP encoding output,
    returns a list of individual paths,
    each of which is a list of states,
    each of which is a tuple of choice + outcomes,
    and separately returns the initial setting'''
    
    f = open(path)
    raw = f.read().replace('\n','')
    raw = raw.split('State 0')[1:]
    
    token = re.compile("\w+\(.*\)")
    
    for story in range(len(raw)):
        raw[story] = raw[story].split('State')
        
        for state in range(len(raw[story])):
            raw[story][state] = raw[story][state].split(' ')
            raw[story][state] = list(filter(None, raw[story][state]))[1:]
            
            if story == 0 & state == 0:
                initial = raw[0][0]
                
            if state != 0:
                choice = None
                outcomes = []
                
                for item in range(len(raw[story][state])):
                    if token.match(raw[story][state][item]):
                        raw[story][state][item] = token.match(raw[story][state][item]).group()
                    if raw[story][state][item][0] == 'd':
                        if choice != None:
                            print('Error! More than one decision in a path state!')
                        choice = raw[story][state][item]
                    elif token.match(raw[story][state][item]):
                        outcomes.append(raw[story][state][item])
                        
                raw[story][state] = (choice, outcomes)
                
        del raw[story][0]
        
    return raw, initial

In [4]:
# HIDDEN

def play(path):
    
    stories, initial = process_output(path)
    
    for item in initial:
        print(item)
    print('')
    
    choose(stories)
    
def progress(stories, decision = None):
    
    
    end = False
    
    # no decision means all paths have same next state (usually story change)
    if decision == None:
        for item in stories[0][0][1]:
            if item == 'story(end)':
                end = True
            print(item)
        print('')
    else: # use decision to print choices
        new_stories = []
        for idx in range(len(stories)):
            if stories[idx][0][0] == decision:
                new_stories.append(stories[idx])
        stories = new_stories
        # next: print outcomes
        for item in stories[0][0][1]:
            if item == 'story(end)':
                end = True
            print(item)
    
    # remove first state for all, continue to choose
    for story in stories:
        del story[0]
        
    if end == True:
        return
    
    choose(stories)
            
def choose(stories):
    # make next decision
    
    # gather choices
    choices = []
    for idx in range(len(stories)):
        choices.append(stories[idx][0][0])
    choices = list(set(choices))
    
    def decide(choice):
        nonlocal decision
        decision = choice
        
    def ready(dummy):
        progress(stories, decision)
    
    # if no choice, automatically progress
    if choices == [None]:
        progress(stories)
    else:        
        decision = choices[0]
        ipywidgets.interact(decide, choice = choices)
        button = Button(description='next')
        display(button)
        button.on_click(ready)
            

    # first: remove all stories where the choice in question is not used:


In [5]:
def go(dummy):
    play('results/01-heist-18.txt')

button = Button(description='heist story')
display(button)
button.on_click(go)

Button(description='heist story', style=ButtonStyle())

In [6]:
# HIDDEN

import nbinteract as nbi
nbi.publish('cramerus/nbinteract-image/master', 'if_generator.ipynb')

# CLI: nbinteract -s cramerus/ASP-Interactive-Fiction-Generator/master if_generator.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'if_generator.ipynb'.
Converting notebook...



Successfully converted!

<a href="if_generator.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
